# 4. EmsembleMean

kaggle score: 0.99357

## Run name

In [1]:
import time

project_name = 'DigitRecognizer'
step_name = 'EmsembleMean'
date_str = time.strftime("%Y%m%d", time.localtime())
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = '%s_%s_%s' % (project_name, step_name, time_str)
print('run_name: %s' % run_name)
t0 = time.time()

run_name: DigitRecognizer_EmsembleMean_20180417_211659


## Import PKGs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import gc
import math
import shutil
import zipfile
import pickle
import h5py
from PIL import Image

from tqdm import tqdm
from multiprocessing import cpu_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

## Basic folders

In [3]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
log_folder = os.path.join(cwd, 'log')
model_folder = os.path.join(cwd, 'model')
output_folder = os.path.join(cwd, 'output')
print('input_folder: \t\t%s' % input_folder)
print('log_folder: \t\t%s' % log_folder)
print('model_folder: \t\t%s' % model_folder)
print('output_folder: \t\t%s'% output_folder)

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
print('\ntrain_csv_file: \t%s' % train_csv_file)
print('test_csv_file: \t\t%s' % test_csv_file)

processed_data_file = os.path.join(input_folder, '%s_%s.p' % (project_name, step_name))
print('processed_data_file: \t%s' % processed_data_file)

input_folder: 		D:\Kaggle\digit-recognizer\input
log_folder: 		D:\Kaggle\digit-recognizer\log
model_folder: 		D:\Kaggle\digit-recognizer\model
output_folder: 		D:\Kaggle\digit-recognizer\output

train_csv_file: 	D:\Kaggle\digit-recognizer\input\train.csv
test_csv_file: 		D:\Kaggle\digit-recognizer\input\test.csv
processed_data_file: 	D:\Kaggle\digit-recognizer\input\DigitRecognizer_EmsembleMean.p


## Basic functions

In [4]:
import sys
def describe(arr):
    print(arr.shape, arr.min(), arr.max(), sys.getsizeof(arr))

In [5]:
def show_data_images(rows, fig_column, y_data, *args):
    columns = len(args)
    figs, axes = plt.subplots(rows, columns, figsize=(rows, fig_column*columns))
    print(axes.shape)  
    for i, ax in enumerate(axes):
        y_data_str = ''
        if type(y_data) != type(None):
            y_data_str =  '_' + str(y_data[i])
        ax[0].set_title('28x28' + y_data_str)
        for j, arg in enumerate(args):
            ax[j].imshow(arg[i])

## Load predict probability files

In [6]:
def is_npy(s):
    return s.startswith('DigitRecognizer_MoreStructure_20180417') and s.endswith('.p')

def is_larger_than(x):
    splits = x[:-2].split('_')
    acc = int(splits[-1])
#     print(acc)
    if acc >= 9990:
        return True
    return False

print(is_larger_than('DigitRecognizer_MoreStructure_20180417_162556_9990.p'))

file_names = os.listdir(model_folder)
file_names = list(filter(is_npy, file_names))
print(len(file_names))

file_names = list(filter(is_larger_than, file_names))
print(len(file_names))

print(file_names)

True
25
12
['DigitRecognizer_MoreStructure_20180417_003649_9990.p', 'DigitRecognizer_MoreStructure_20180417_011627_9990.p', 'DigitRecognizer_MoreStructure_20180417_043418_9990.p', 'DigitRecognizer_MoreStructure_20180417_063232_9990.p', 'DigitRecognizer_MoreStructure_20180417_071204_9990.p', 'DigitRecognizer_MoreStructure_20180417_095031_9990.p', 'DigitRecognizer_MoreStructure_20180417_103009_10000.p', 'DigitRecognizer_MoreStructure_20180417_122854_9990.p', 'DigitRecognizer_MoreStructure_20180417_130832_9990.p', 'DigitRecognizer_MoreStructure_20180417_150634_10000.p', 'DigitRecognizer_MoreStructure_20180417_154615_9990.p', 'DigitRecognizer_MoreStructure_20180417_162556_9990.p']


In [7]:
def load_proba(file_name):
    with h5py.File(file_name, 'r') as h:
        y_data_proba = np.array(h['y_data_proba'])
        y_data = np.array(h['y_data'])
        y_test_proba = np.array(h['y_test_proba'])
    print('File loaded: \t%s' % file_name)
    
    return y_data_proba, y_data, y_test_proba


y_proba_file = os.path.join(model_folder, file_names[0])
y_data_proba, y_data, y_test_proba = load_proba(y_proba_file)

print(y_data_proba.shape)
print(y_data.shape)
print(y_test_proba.shape)

File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_003649_9990.p
(42000, 10)
(42000, 10)
(28000, 10)


## Mean

In [8]:
from sklearn.metrics import accuracy_score

y_proba_file = os.path.join(model_folder, file_names[0])
y_data_probas, y_datas, y_test_probas = load_proba(y_proba_file)

print(y_data_probas.shape)
print(y_datas.shape)
print(y_test_probas.shape)

for file_name in file_names[1:]:
    file_path = os.path.join(model_folder, file_name)
    y_data_proba, y_data, y_test_proba = load_proba(file_path)
    y_data_probas = y_data_probas + y_data_proba
    y_test_probas = y_test_probas + y_test_proba
    
y_data_probas = y_data_probas / len(file_names)
y_test_probas = y_test_probas / len(file_names)

print(len(y_data_probas))
print(y_data_probas[:2])

File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_003649_9990.p
(42000, 10)
(42000, 10)
(28000, 10)
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_011627_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_043418_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_063232_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_071204_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_095031_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_103009_10000.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_122854_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_MoreStructure_20180417_130832_9990.p
File loaded: 	D:\Kaggle\digit-recognizer\model\DigitRecognizer_

## Predict

In [9]:
y_data_pred = np.argmax(y_data_probas, axis=1)
print('y_test_pred.shape: ', y_data_pred.shape)
y_data = np.argmax(y_data, axis=1)

data_acc = accuracy_score(y_data, y_data_pred)

y_test_pred.shape:  (42000,)


In [10]:
run_name_acc = run_name + '_' + str(int(data_acc*10000)).zfill(4)
print(run_name_acc)

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
pred_file = os.path.join(output_folder, run_name_acc + '.csv')
print(pred_file)

DigitRecognizer_EmsembleMean_20180417_211659_9993
D:\Kaggle\digit-recognizer\output\DigitRecognizer_EmsembleMean_20180417_211659_9993.csv


In [11]:
y_test_pred = np.argmax(y_test_proba, axis=1)
print('y_test_pred.shape: ', y_test_pred.shape)

with open(pred_file, 'w') as f:
    f.write('ImageId,Label\n')
    for i in range(len(y_test_pred)) :
        f.write("".join([str(i+1),',',str(y_test_pred[i]),'\n']))

y_test_pred.shape:  (28000,)


In [12]:
print(run_name_acc)

t1 = time.time()
print('time cost: %.2f s' % (t1-t0))
print('Done!')

DigitRecognizer_EmsembleMean_20180417_211659_9993
time cost: 2.67 s
Done!
